In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from google.colab import drive
# drive.mount('/content/drive') ['Corrective','Adaptive','Perfective']
label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}

In [3]:
df = pd.read_csv(r'Commit_dataset.csv',encoding='cp1252')
df = df.replace({"3_labels": label2id})

In [4]:
df

,commit_id,project,comment,3_labels,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
0,0531b8bff5c14d9504beefb4ad47f473e3a22932,ReactiveX-RxJava,Change hasException to hasThrowable--,Perfective,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0950c46beda335819928585f1262dfe1dca78a0b,ReactiveX-RxJava,Trying to extend the Scheduler interface accor...,Adaptive,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0f92fdd8e6422d5b79c610a7fd8409d222315a49,ReactiveX-RxJava,RunAsync method for outputting multiple values--,Adaptive,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100f571c9a2835d5a30a55374b9be74c147e031f,ReactiveX-RxJava,forEach with Action1 but not Observer--I re-re...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,191f023cf5253ea90647bc091dcaf55ccdce81cc,ReactiveX-RxJava,1.x: Fix Completable swallows- OnErrorNotImple...,Corrective,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,51e9da224f80254476a7dc446bca817b505381d8,jenkinsci$clearcase-plugin,Use a temporary file to decrease memory consum...,Perfective,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1789,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,jexp$batch-import,added a more memory efficient structure for st...,Perfective,0,0,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1790,19b650c78a1c76f4fd90274d7f163f863c0d39e4,hdiv$hdiv,Memory and performance optimizations,Perfective,0,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
1791,d7bf95159df37a3d338ca267dddd3d26b38ec37c,casidiablo$persistence,Now it is possible to specify the sqlite open ...,Perfective,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df = df.rename(columns={'3_labels':'label','comment':'text'})
train, test = train_test_split(df,test_size=0.15,stratify=df['label'],random_state=1)

In [6]:
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [7]:
# train = df.rename(columns={'3_labels':'label','comment':'text'})

In [8]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)


In [9]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [10]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [11]:
len(train)

1524

In [12]:
# encoded_train = tokenizer(train_dataset['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_train["input_ids"].shape)
# encoded_test = tokenizer(test['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_test["input_ids"].shape)
# # encoded_val = tokenizer(val['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding='max_length')

In [13]:
# encoded_train

In [14]:
train['label'].value_counts()

label
Corrective    513
Perfective    510
Adaptive      501
Name: count, dtype: int64

In [15]:
test['label'].value_counts()

label
Corrective    90
Perfective    90
Adaptive      89
Name: count, dtype: int64

In [16]:
len(train_dataset)

1548

In [17]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [18]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [19]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [20]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [21]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [22]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:02<00:00,  7.85it/s]
***** Running training *****
  Num examples = 61920
  Num epochs = 1
  Total optimization steps = 3870
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [13:56<00:00, 836.82s/it]
***** Running evaluation *****


CPU times: user 18min 21s, sys: 5min 42s, total: 24min 4s
Wall time: 14min 2s


{'accuracy': 0.8141263940520446,
 'precision': 0.814045908655574,
 'recall': 0.8141263940520446,
 'f1': 0.814035564829199}

In [23]:
trainer.model.save_pretrained('my-awesome-model')

In [24]:
train.to_csv('train.csv')

In [25]:
test.to_csv('test.csv')